In [3]:
import requests
from PIL import Image
import os

# API anahtarınızı doğrudan koda yazın
apiKey = 'dcbe55c1e3134568a5824acb8fc1ca1a'
baseUrl = "http://api.openweathermap.org/data/2.5/"

# İngilizce hava durumu açıklamalarının Türkçe karşılıkları
aciklamaCevirisi = {
    'clear sky': 'açık gökyüzü',
    'few clouds': 'az bulutlu',
    'scattered clouds': 'dağınık bulutlar',
    'broken clouds': 'kısmen bulutlu',
    'shower rain': 'kısa süreli sağanak',
    'rain': 'yağmur',
    'thunderstorm': 'gök gürültülü fırtına',
    'snow': 'kar',
    'mist': 'sis',
    'haze': 'pus',
    'fog': 'kısa mesafe görüş',
    'dust': 'toz',
    'sand': 'kum',
    'squall': 'şiddetli rüzgar'
}

# Türkçe hava durumu açıklamalarının görsel dosya isimleri
gorselYolu = {
    'açık gökyüzü': 'gunesli.png',
    'az bulutlu': 'bulutlu.png',
    'dağınık bulutlar': 'brokenclouds.png',
    'kısmen bulutlu': 'brokenclouds.png',
    'kısa süreli sağanak': 'showerrain.png',
    'yağmur': 'yagmurlu.png',
    'gök gürültülü fırtına': 'firtinali.png',
    'kar': 'karli.png',
    'sis': 'sisli.png',
    'püs': 'sisli.png',
    'kısa mesafe görüş': 'sisli.png',
    'toz': 'default.png',
    'kum': 'default.png',
    'şiddetli rüzgar': 'default.png'
}

class HavaDurumuAlici:
    def __init__(self, apiKey):
        self.apiKey = apiKey

    def havaDurumuGetir(self, sehir):
        try:
            url = f"{baseUrl}weather?q={sehir}&appid={self.apiKey}&units=metric"
            cevap = requests.get(url)
            veri = cevap.json()

            if cevap.status_code == 200:
                return veri
            else:
                print(f"{sehir} şehri bulunamadı veya başka bir hata oluştu. Durum kodu: {cevap.status_code}.")
                return None
        except Exception as e:
            print(f"Hata: {e}")
            return None

    def tahminGetir(self, sehir):
        try:
            url = f"{baseUrl}forecast?q={sehir}&appid={self.apiKey}&units=metric"
            cevap = requests.get(url)
            veri = cevap.json()

            if cevap.status_code == 200:
                return veri
            else:
                print(f"{sehir} şehri bulunamadı veya başka bir hata oluştu. Durum kodu: {cevap.status_code}.")
                return None
        except Exception as e:
            print(f"Hata: {e}")
            return None

    def havaDurumuBilgileriniYazdir(self, sehir):
        havaDurumuVerisi = self.havaDurumuGetir(sehir)

        if havaDurumuVerisi:
            print(f"\n{sehir} için hava durumu:")
            print(f"Sıcaklık: {havaDurumuVerisi['main']['temp']}°C")
            print(f"Hissedilen Sıcaklık: {havaDurumuVerisi['main']['feels_like']}°C")
            print(f"Nem: {havaDurumuVerisi['main']['humidity']}%")
            print(f"Rüzgar Hızı: {havaDurumuVerisi['wind']['speed']} m/s")
            print(f"Rüzgar Yönü: {havaDurumuVerisi['wind']['deg']}°")

            if 'weather' in havaDurumuVerisi and len(havaDurumuVerisi['weather']) > 0:
                aciklamaIng = havaDurumuVerisi['weather'][0]['description']
                aciklamaTurkce = aciklamaCevirisi.get(aciklamaIng.lower(), 'bilinmeyen hava durumu')
                print(f"Hava Durumu: {aciklamaTurkce.capitalize()}")

                # Görseli göstermek için metot çağır
                self.gorselGoster(aciklamaTurkce)
            else:
                print("Hava durumu bilgisi bulunamadı.")
    
    def farkliZamanDilimlerindeHavaDurumu(self, sehir):
        tahminVerisi = self.tahminGetir(sehir)
        if tahminVerisi:
            print(f"\n{sehir} için farklı zaman dilimlerinde hava durumu:")
            for giris in tahminVerisi['list']:
                tarih = giris['dt_txt']
                sicaklik = giris['main']['temp']
                aciklamaIng = giris['weather'][0]['description']
                aciklamaTurkce = aciklamaCevirisi.get(aciklamaIng.lower(), 'bilinmeyen hava durumu')
                print(f"Tarih: {tarih} - Sıcaklık: {sicaklik}°C - Hava Durumu: {aciklamaTurkce.capitalize()}")

    def gorselGoster(self, aciklama):
        # Görsel dosyasını belirle
        gorselAdi = gorselYolu.get(aciklama.lower(), 'default.png')
        
        # Görsel yolunu oluştur
        gorselYoluTam = os.path.join('images', gorselAdi)
        
        # Dosya var mı kontrol et
        if os.path.exists(gorselYoluTam):
            try:
                # Görseli yükle
                img = Image.open(gorselYoluTam)
                img.show()
            except Exception as e:
                print(f"Görsel yüklenirken başka bir hata oluştu: {e}")
        else:
            print(f"Görsel yüklenirken hata oluştu: {gorselYoluTam} dosyası bulunamadı.")

def ana():
    print("Hava Durumu Uygulamasına Hoşgeldiniz!")
    print("Bu program, girdiğiniz şehir için hava durumu bilgilerini, farklı zaman dilimlerinde hava durumunu ve hava durumu uyarılarını sağlar.")
    print("Programdan çıkmak için 'çık' yazabilirsiniz.\n")
    
    alici = HavaDurumuAlici(apiKey)
    
    while True:
        sehir = input("Şehir adını girin (çıkmak için 'çık' yazın): ").strip()
        if sehir.lower() == 'çık':
            print("Programdan çıkılıyor...")
            break
        if not sehir:
            print("Geçersiz şehir adı, lütfen tekrar deneyin.")
            continue

        alici.havaDurumuBilgileriniYazdir(sehir)
        alici.farkliZamanDilimlerindeHavaDurumu(sehir)

if __name__ == "__main__":
    ana()

Hava Durumu Uygulamasına Hoşgeldiniz!
Bu program, girdiğiniz şehir için hava durumu bilgilerini, farklı zaman dilimlerinde hava durumunu ve hava durumu uyarılarını sağlar.
Programdan çıkmak için 'çık' yazabilirsiniz.


ankara için hava durumu:
Sıcaklık: 25.11°C
Hissedilen Sıcaklık: 24.77°C
Nem: 42%
Rüzgar Hızı: 1.34 m/s
Rüzgar Yönü: 33°
Hava Durumu: Açık gökyüzü
Görsel yüklenirken hata oluştu: images\gunesli.png dosyası bulunamadı.

ankara için farklı zaman dilimlerinde hava durumu:
Tarih: 2024-08-12 21:00:00 - Sıcaklık: 25.11°C - Hava Durumu: Açık gökyüzü
Tarih: 2024-08-13 00:00:00 - Sıcaklık: 23.44°C - Hava Durumu: Açık gökyüzü
Tarih: 2024-08-13 03:00:00 - Sıcaklık: 20.8°C - Hava Durumu: Açık gökyüzü
Tarih: 2024-08-13 06:00:00 - Sıcaklık: 23.14°C - Hava Durumu: Açık gökyüzü
Tarih: 2024-08-13 09:00:00 - Sıcaklık: 28.39°C - Hava Durumu: Açık gökyüzü
Tarih: 2024-08-13 12:00:00 - Sıcaklık: 31.46°C - Hava Durumu: Açık gökyüzü
Tarih: 2024-08-13 15:00:00 - Sıcaklık: 31.8°C - Hava Durumu: Aç